In [ ]:

import kagglehub
jainpooja_fake_news_detection_path = kagglehub.dataset_download('jainpooja/fake-news-detection')

print('Data source import complete.')


Data source import complete.


# Fake News Detection

![image.png](attachment:image.png)

## Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

## Importing Dataset

In [ ]:
df_fake = pd.read_csv("/content/sample_data/Fake.csv")
df_true = pd.read_csv("/content/sample_data/True.csv")

In [ ]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [ ]:
df_true.head(5)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


## Inserting a column "class" as target feature

In [ ]:
df_fake["class"] = 0
df_true["class"] = 1

In [ ]:
df_fake.shape, df_true.shape

((23481, 5), (21417, 5))

## Merging True and Fake Dataframes

In [ ]:
df_merge = pd.concat([df_fake, df_true], axis = 0 )
df_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [ ]:
df_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

## Removing columns which are not required

In [ ]:
df = df_merge.drop(["title", "subject","date"], axis = 1)

In [ ]:
df.isnull().sum()

,0
text,0
class,0


## Random Shuffling the dataframe

In [ ]:
df.columns

Index(['text', 'class'], dtype='object')

In [ ]:
df.head()

,text,class
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0


In [ ]:
news=df[df['class']==1]['text'][100]
print(news)

WASHINGTON (Reuters) - Democratic Senator Elizabeth Warren is taking aim at budget chief Mick Mulvaney’s plan to fill the ranks of the U.S. consumer financial watchdog with political allies, according to letters seen by Reuters, the latest salvo in a broader battle over who should run the bureau. President Donald Trump last month appointed Mulvaney as acting director of the Consumer Financial Protection Bureau (CFPB), though the decision is being legally challenged by the agency’s deputy director, Leandra English, who says she is the rightful interim head. Mulvaney told reporters earlier this month he planned to bring in several political appointees to help overhaul the agency, but Warren warned in a pair of letters sent Monday to Mulvaney and the Office of Personnel Management (OPM), which oversees federal hiring, that doing so was inappropriate and potentially illegal. The CFPB is meant to be an independent agency staffed primarily by non-political employees. Hiring political appoint

## Creating a function to process the texts

In [ ]:
import re
import string

def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W", " ", text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

# Sample text for testing
sample_text = """
An AI-generated image depicting tent camps for displaced Palestinians and a slogan that reads All Eyes on Rafah is sweeping social media.
The post has been shared more than 47 million times by Instagram users including celebrities like Dua Lipa, Lewis Hamilton and Gigi and Bella Hadid.
The image and the slogan went viral after an Israeli air strike and resulting fire at a camp for displaced Palestinians in the southern Gaza city of Rafah earlier this week.
The Hamas-run health ministry said at least 45 people were killed and hundreds more wounded in the incident. Israel said it had targeted two Hamas commanders, and that the deadly fire was possibly caused by a secondary explosion.
There has been widespread international condemnation of the Israeli strike, which Israeli Prime Minister Benjamin Netanyahu called a “tragic mishap”.


1:00
Video shows devastation in camp for displaced Palestinians after Israeli strike
Following the incident, the image went viral after a young man in Malaysia posted it on his social media account, BBC Arabic discovered.
So how did this image come about?
Where did the slogan All Eyes on Rafah start?
The deadly incident earlier this week in Rafah led to people posting clips of Richard Peeperkorn, a representative of the World Health Organization in the occupied Palestinian territories, speaking in February.
He told journalists at the time that “All eyes are on Rafah", warning against Israeli forces attacking the city.
United Nations Richard Peeperkorn addressing journalists over a weblink from GazaUnited Nations
Richard Peeperkorn told journalists at the UN in February that "All eyes are on Rafah"
Speaking via an online call to journalists at the UN's headquarters in Geneva, Mr Peeperkorn said he feared "unimaginable disaster" if the Israeli army made a large-scale incursion into the city, which it was threatening to invade.
Officials and activists have been repeating Mr Peeperkorn's phrase since then to express their concern and opposition to Israel's military operation in Rafah, which began three weeks ago.
Reuters Students occupy the street in front of the Sciences Po University building in support of Palestinians in Gaza, amid the ongoing conflict between Israel and the Palestinian Islamist group Hamas, in Paris, France, May 7, 2024. The slogan reads "All eyes on Rafah". REUTERS/Johanna GeronReuters
Students occupy the street in front of the Sciences Po University building in Paris, with a sign saying All Eyes on Rafah
In the months that followed the first use of the phrase, All Eyes on Rafah slogans appeared at rallies around the world and on social media.
But in the last two days, the AI-generated image featuring the slogan has proliferated on social media sites, with more than 47 million shares according to an Instagram count on Thursday afternoon.
Other celebrities who have shared the image and the slogan include American actor Mark Ruffalo, Indian actress Priyanka Chopra and Syrian actress Kinda Alloush.
How did the photo spread across social media in the past two days?
Experts who spoke to the BBC say there are a number of factors that explain why the All Eyes on Rafah message has gone viral in such a short amount of time.
Among them are the AI-generated nature of the image, the simplicity of the slogan, the ease at which Instagram users can share the post in just a couple of clicks, and its uptake by celebrities.
But according to Anastasia Kavada, who runs an MA course on media, campaigning and social change at the University of Westminster, the most important factor is the timing and political context of the post.
She explains that it has gone viral at a time when many feel "outraged" at the news of the strike on the camp in Rafah.
This view is echoed by Maher Nammari, an e-marketing and artificial intelligence consultant, who emphasises the importance of the nature of the incident itself and the online interaction that followed.
"Sunday's attack in Rafah has caused widespread sadness", he told BBC Arabic.
shahv4012/Instagram An AI-generated image shows orderly lines of tents in rows with mountains in the background. The words 'All Eyes on Rafah' features on top of the tents.shahv4012/Instagram
Has AI helped take the post viral?
What makes it more shareable still, however, is the AI-generated image in the post itself, experts say.
The image - depicting a vast desert and a tent camp with the text All Eyes on Rafah on it - does not show a real place or the city of Rafah.
Notably absent are pictures of dead bodies, blood, shots of real people, names or distressing scenes.
This has led to criticism from some quarters.
Dr Paul Reilly, a senior lecturer in Communications, Media and Democracy at the University of Glasgow, says that some activists will feel concerned that the image does not show the true extent of what's really happening on the ground, pointing to material posted on social media by journalists in Gaza which has not gained such viral prominence.
Reacting to the criticism, the Malaysian artist who created the image - known by his username shahv4012 - wrote in an Instagram story quoted by Time magazine: “There are people who are not satisfied with the picture and template, I apologize if I have made a mistake on all of you."
But he added: “Whatever [you do], don’t look down on the Rafah issue now, spread it so that they are shaken and afraid of the spread of all of us.”
But while the image depicts a somewhat "sanitised" version of what is going on in Rafah, Dr Reilly says that could be an advantage from a digital activist's perspective in terms of its share-ability.
He explains that this is because the image does not contain graphic content that could lead to its removal from Instagram for infringing its terms of use guidelines, while also increasing awareness of the issue that activists seek to bring attention to.
"""""
# Process the sample text
processed_text = wordopt(sample_text)
print(processed_text)


 an ai generated image depicting tent camps for displaced palestinians and a slogan that reads all eyes on rafah is sweeping social media  the post has been shared more than  million times by instagram users including celebrities like dua lipa  lewis hamilton and gigi and bella hadid  the image and the slogan went viral after an israeli air strike and resulting fire at a camp for displaced palestinians in the southern gaza city of rafah earlier this week  the hamas run health ministry said at least  people were killed and hundreds more wounded in the incident  israel said it had targeted two hamas commanders  and that the deadly fire was possibly caused by a secondary explosion  there has been widespread international condemnation of the israeli strike  which israeli prime minister benjamin netanyahu called a  tragic mishap       video shows devastation in camp for displaced palestinians after israeli strike following the incident  the image went viral after a young man in malaysia pos

In [ ]:
df["text"] = df["text"].apply(wordopt)

## Defining dependent and independent variables

In [ ]:
x = df["text"]
y = df["class"]

## Splitting Training and Testing

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, shuffle=True,stratify=y)
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5, shuffle=True)

## Convert text to vectors

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)
xv_val = vectorization.transform(x_val)

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [ ]:
pred_lr=LR.predict(xv_test)

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

mae=mean_absolute_error(pred_lr,y_test)
mse=mean_squared_error(pred_lr,y_test)
print(mae,mse) #least the better

0.014965259219668627 0.014965259219668627


In [ ]:
LR.score(xv_test, y_test) #more the better

0.9850347407803314

In [ ]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      2889
           1       0.98      0.99      0.98      2724

    accuracy                           0.99      5613
   macro avg       0.98      0.99      0.99      5613
weighted avg       0.99      0.99      0.99      5613



## Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

GBC = GradientBoostingClassifier()
GBC.fit(xv_train, y_train)

GradientBoostingClassifier()

In [ ]:
pred_gbc = GBC.predict(xv_test)

In [ ]:
GBC.score(xv_test, y_test)

In [ ]:
#print(score)

In [ ]:
print(classification_report(y_test, pred_gbc))

## Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier(n_estimators=100)
RFC.fit(xv_train, y_train)

NameError: name 'xv_train' is not defined

In [ ]:
pred_rfc = RFC.predict(xv_test)

In [ ]:
RFC.score(xv_test, y_test)

NameError: name 'xv_test' is not defined

In [ ]:
print(classification_report(y_test, pred_rfc))

## Model Testing

In [ ]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"

def manual_testing(news):
    print(news)
    testing_news = {"text":[news]}
    print(testing_news)
    new_def_test = pd.DataFrame(testing_news)
    print("\n",new_def_test.head())
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    print(new_def_test.head())
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    print(new_xv_test)
    pred_LR = LR.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_lable(pred_LR[0]),output_lable(pred_GBC[0]),output_lable(pred_RFC[0])))


In [ ]:
#import re
#import string

#def wordopt(text):
 #   text = text.lower()
  #  text = re.sub('\[.*?\]', '', text)
   # text = re.sub("\\W"," ",text)
    #text = re.sub('https?://\S+|www\.\S+', '', text)
    #text = re.sub('<.*?>+', '', text)
    #text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
#    text = re.sub('\n', '', text)
 #   text = re.sub('\w*\d\w*', '', text)
  #  return text

# Sample text for testing
#sample_text = """

#In the coming months, more details about the project are expected to emerge, providing further insight into the technical and financial aspects of Space Force One. For now, the announcement has succeeded in keeping Trump at the center of media attention, reinforcing his reputation for unconventional and headline-grabbing campaign tactics.
#"""
# Process the sample text
#processed_text = wordopt(sample_text)
#print(processed_text)


In [ ]:
news=processed_text
manual_testing(news)

NameError: name 'processed_text' is not defined

Scope of the Project (in % terms):

Text Preprocessing & Cleaning – 20%

Feature Extraction using TF-IDF – 20%

Model Building & Training (e.g., Logistic Regression / Gradient Boosting) – 30%

Evaluation & Testing – 15%

Visualization (e.g., WordCloud, confusion matrix) – 10%

Future Scope & Improvements (e.g., larger dataset, web app) – 5%

Total: 100%